all_data 所有資料
authors_data->可以放新一批人取抓轉發
生成new

In [1]:
# 從首頁抓
import csv
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Set up Selenium WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.threads.net/?hl=zh-tw")
script_directory = os.getcwd()

# Wait for the page to load (adjust timeout if needed)
driver.implicitly_wait(10)

# Scroll and find elements
SCROLL_PAUSE_TIME = 2
SCROLL_TIME_LIMIT = 100  # Time limit for scrolling in seconds
WRITE_INTERVAL = 10  # Write to CSV every 10 seconds
MAX_TOTAL_TIME = 30  # Max time limit for scraping replies in seconds
start_time = time.time()
last_write_time = start_time  # Store the time of last write

# Specify the full path to the CSV file
csv_file_path = os.path.join(script_directory, 'authors_data.csv')
print(f"Saving CSV to: {csv_file_path}")

# Open CSV file for writing with UTF-8 encoding
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # Write the header
    writer.writerow(['user'])

    # Set to store unique users
    users_to_write = set()  # Using set to avoid duplicates

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Check if time limit has been reached
        elapsed_time = time.time() - start_time
        if elapsed_time > SCROLL_TIME_LIMIT:
            break

        # Find elements (adjust selectors based on actual content)
        author_elements = driver.find_elements(By.CLASS_NAME, 'x1pha0wt')

        # Print out the number of elements found
        print(f"Found {len(author_elements)} author elements")

        # Iterate over each element and collect user data
        for element in author_elements:
            user = element.text.split('\n')[0].strip()  # Extract user text (first line)

            # Debugging step: print user
            print("Found user:", user)

            # Collect user data in the set (avoids duplicates)
            users_to_write.add(user)

        # Write to CSV every 10 seconds
        if time.time() - last_write_time >= WRITE_INTERVAL:
            # Write all collected unique users to CSV
            for user in users_to_write:
                writer.writerow([user])  # Write user data to CSV
            print(f"Written {len(users_to_write)} unique users to CSV.")
            
            # Reset the set of users and update last write time
            users_to_write.clear()
            last_write_time = time.time()

        # Delay before next scroll
        time.sleep(1)  # Small delay to ensure smooth scrolling

    print("CSV file created successfully.")


driver.quit()

Saving CSV to: C:\Users\sandy\Desktop\IRTM\authors_data.csv
Found 15 author elements
Found user: liiiiiiiiiiji
Found user: _4in
Found user: 77witch_7two
Found user: qq_babysoda_
Found user: nnonimo_
Found user: benlinmarketing
Found user: vicohsu
Found user: jolyne.ovo
Found user: _____8.14
Found user: qq_babysoda_
Found user: nanitori
Found user: shirley.lyn
Found user: yurong_ccc1107
Found user: __helenka__
Found user: liaobohao
Found 25 author elements
Found user: liiiiiiiiiiji
Found user: _4in
Found user: 77witch_7two
Found user: qq_babysoda_
Found user: nnonimo_
Found user: benlinmarketing
Found user: vicohsu
Found user: jolyne.ovo
Found user: _____8.14
Found user: qq_babysoda_
Found user: nanitori
Found user: shirley.lyn
Found user: yurong_ccc1107
Found user: __helenka__
Found user: liaobohao
Found user: doraelun1978
Found user: nikkilyxx
Found user: ice_i_eat
Found user: imf_502
Found user: ettodaypets
Found user: swinghong_0618
Found user: ruru1419
Found user: kevin_ooxx
Found 

In [1]:
# 從轉發抓
import csv
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Set up Selenium WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.threads.net/?hl=zh-tw")
script_directory = os.getcwd()

# Wait for the page to load (adjust timeout if needed)
driver.implicitly_wait(10)

# CSV file paths
original_csv_file_path = os.path.join(script_directory, 'authors_data.csv')
new_csv_file_path = os.path.join(script_directory, 'new_authors_data.csv')

# Function to read all users from the original CSV
def read_users_from_csv():
    users = []
    try:
        with open(original_csv_file_path, mode='r', newline='', encoding='utf-8') as file:
            reader = csv.reader(file)
            next(reader)  # Skip header row
            for row in reader:
                users.append(row[0])  # Get user from the first column
    except FileNotFoundError:
        print(f"CSV file {original_csv_file_path} not found.")
    return users

# Function to process the profiles of users and save new data to a separate file
def process_user_profiles(users_to_process):
    with open(new_csv_file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Write the header to the new CSV
        writer.writerow(['user', 'origin'])

        # Loop through the collected users and visit their profile pages
        for user in users_to_process:
            profile_url = f"https://www.threads.net/@{user}?hl=zh-tw"
            
            # Navigate to the user's profile
            driver.get(profile_url)
            time.sleep(3)  # Wait for the page to load

            try:
                # Find the reply button (aria-label="轉發")
                reply_button = driver.find_element(By.XPATH, "//*[@aria-label='轉發']")
                
                # Click the reply button to continue scraping (this might open the replies section or show more content)
                reply_button.click()
                print(f"Clicked '轉發' on {user}'s profile.")
                
                # Wait for new content to load after clicking (adjust as necessary)
                time.sleep(3)
                
                # Now, scroll to the bottom of the reply section
                reply_start_time = time.time()  # Track the time spent in the reply section
                last_height = driver.execute_script("return document.body.scrollHeight")  # Get the initial scroll height
                while time.time() - reply_start_time < 30:  # MAX_TOTAL_TIME is 30 seconds
                    # Scroll down
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(2)  # SCROLL_PAUSE_TIME

                    # Get the new scroll height after scrolling
                    new_height = driver.execute_script("return document.body.scrollHeight")

                    # Check if we have reached the bottom (no more new content)
                    if new_height == last_height:
                        print("Reached the bottom of the reply section.")
                        break

                    last_height = new_height  # Update last_height for the next iteration

                # Collect new users from the reply section
                reply_author_elements = driver.find_elements(By.CLASS_NAME, 'x1pha0wt')
                print(f"Found {len(reply_author_elements)} reply author elements")

                # Add the new users from the reply section to the new CSV and print origin
                for element in reply_author_elements:
                    reply_user = element.text.split('\n')[0].strip()  # Extract user text (first line)
                    print(f"Found reply user: {reply_user} from {user}'s reply section.")
                    writer.writerow([reply_user, f"Reply from {user}"])  # Save new user with origin

            except Exception as e:
                print(f"Could not click the reply button for {user}: {e}")

# Read all users from the original CSV file
users_in_csv = read_users_from_csv()

# Process the profiles of users read from the CSV and save new data
process_user_profiles(users_in_csv)

# Close the browser
driver.quit()

print(f"New data saved to {new_csv_file_path}")


Clicked '轉發' on taylornation's profile.
Reached the bottom of the reply section.
Found 3 reply author elements
Found reply user: taylorswift from taylornation's reply section.
Found reply user: sammisrepera from taylornation's reply section.
Found reply user: enchanted2meetu from taylornation's reply section.
Clicked '轉發' on yijing5252's profile.
Reached the bottom of the reply section.
Found 25 reply author elements
Found reply user: yisan0519 from yijing5252's reply section.
Found reply user: ness.wellness from yijing5252's reply section.
Found reply user: _unix.l from yijing5252's reply section.
Found reply user: holiao_comes from yijing5252's reply section.
Found reply user: x__x__x0101 from yijing5252's reply section.
Found reply user: fpcchannel.tomi from yijing5252's reply section.
Found reply user: hsioufeng.chen from yijing5252's reply section.
Found reply user: read.chouuu from yijing5252's reply section.
Found reply user: sherrys_notes from yijing5252's reply section.
Found 